In [11]:
import numpy as np
import numpy.linalg as lin
import numpy.random as npr 
import scipy.optimize as sco
import scipy.stats as si 
import time
from scipy import interpolate 
from scipy.linalg import expm
from scipy.fftpack import fft, ifft
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'

In [12]:
# EUR-A & MJD-A
S0 = 100
G = 100
T = 10
M = 10
r = 0.05
sigma = 0.15
m = 0.02
lamb = 0.1
mu = -1.08
nu = 0.4

N, X = 2048, 7.5 # FST param.
I = 1000000  # Monte Carlo param.

In [13]:
def mjd_mc(S0, G, T, r, sigma, m, lamb, mu, nu, M, I):
    dt = T / M
    kappa = np.exp(mu + 0.5 * nu ** 2) - 1
    drift = r - m - lamb * kappa - 0.5 * sigma ** 2
    S = np.zeros((M + 1, I))
    S[0] = S0
    for s in range(I):
        for t in range(1, M + 1):
            J = 0
            Nt = npr.poisson(lamb * dt)
            if Nt > 0:
                for i in range(Nt):
                    J = J + npr.normal(mu, nu)
            Z = npr.standard_normal()
            S[t, s] = S[t-1, s] * np.exp(drift * dt + sigma * np.sqrt(dt) * Z + J)
    pT = np.maximum(G - S[-1], 0)
    cT = np.maximum(S[-1] - G, 0)
    P0 = np.exp(-r * T) * np.mean(pT)
    C0 = np.exp(-r * T) * np.mean(cT)
    return (P0, C0)
print(mjd_mc(S0, G, T, r, sigma, m, lamb, mu, nu, M, I)[0]) # 18.000379476851418
print(mjd_mc(100, 100, 0.25, 0.05, 0.15, 0, 0.1, -0.9, 0.45, M, I)[1]) # 4.392337647703946

17.999561829439603
4.389167403431759


In [10]:
def mjd_fst(S0, G, T, r, sigma, m, lamb, mu, nu, N, X):
    # Real space
    x_min, x_max = -X, X
    dx = (x_max - x_min) / (N - 1)
    x = np.linspace(x_min, x_max, N)
    # Fourier space
    epsilon = 0.0001
    w_max = np.pi / dx
    dw = 2 * w_max / N
    w = np.hstack((np.arange(0, w_max + epsilon, dw), np.arange(-w_max + dw, -dw + epsilon, dw)))
    # Payoff function at time T
    ST = S0 * np.exp(x)
    payoff_p = np.maximum(G - ST, 0)
    payoff_c = np.maximum(ST - G, 0)
    # Matrix characteristic funciton
    rj = lamb * (np.exp(mu + 0.5 * nu ** 2) - 1) 
    psi = 1j * (r - m - rj - 0.5 * sigma ** 2) * w - 0.5 * (sigma * w) ** 2 + lamb * (np.exp(1j * mu * w - 0.5 * (nu * w) ** 2) - 1) - r
    char = np.exp(psi * T) # char_exp_factor
    # FST method
    p = ifft(fft(payoff_p) * char).real
    c = ifft(fft(payoff_c) * char).real
    # Interpolate prices
    fp = interpolate.PchipInterpolator(ST, p)
    P = fp(S0)
    fc = interpolate.PchipInterpolator(ST, c)
    C = fc(S0)
    return (float(P), float(C))
print(mjd_fst(S0, G, T, r, sigma, m, lamb, mu, nu, N, X)[0]) # 18.000379476851418
print(mjd_fst(100, 100, 0.25, 0.05, 0.15, 0, 0.1, -0.9, 0.45, N, X)[1]) # 4.392337647703946

18.000379476851418
4.392337647703946
